<a href="https://colab.research.google.com/github/noemigarcia27/SImulacion-II/blob/main/Linea_espera_2servidores_en_serie.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Librerias que vamos a ocupar

In [35]:
import numpy as np
import matplotlib.pyplot as plt
from random import expovariate
from math import inf as infinity

**Parametros**

In [36]:
lambda_ll = 5.0 #tasa de llegadas
mu_1 = 6.0 #tasa de servicio servidor 1
mu_2 = 7.0 #tasa de servicio servidor 2

Max_ll = 10000 #numero maximo de eventos

Inicializacion del sistema

In [37]:
t = 0 #tiempo actual
n1 = 0 #numero de clientes en el servidor 1
n2 = 0 #numero de clientes en el servidor 2
NLL = 0 #numero total de llegadas
NS = 0  #numero total de salidas

In [38]:
LL_1 = [] #tiempos de llegada
LL = [] # Tiempos de llegada a la COLA del Servidor 2
S = [] #tiempos de salida

Generar $T_0$, es decir la primer llegada

In [39]:
T_0 = expovariate(lambda_ll)
t_ll = T_0
t_1 = infinity #tiempo de servicio1
t_2 = infinity #tiempo de servicio 2

Listas de Salida

In [40]:
t_evento_anterior = 0.0 #Tiempo del
servidor_1 = []
servidor_2 = []

tll_Data = [] #Tiempos de llegada
ts_Data = [] #tiempos de salida final

Funciones auxiliares para generar tiempos

In [41]:
def Generar_T_t():
  return expovariate(lambda_ll)

def Generar_Y():
  return expovariate(mu_1)

def Generar_Y2():
  return expovariate(mu_2)

Iniciamos en el blucle de la simulacion

In [42]:
while NLL < Max_ll:
  proximo_evento = min(t_ll, t_1, t_2)

  if proximo_evento == infinity:
    break

  duracion = proximo_evento - t

  servidor_1.append((n1, duracion))
  servidor_2.append((n2, duracion))

  #Caso 1
  if (t_ll <= t_1) and (t_ll <= t_2):
    t = t_ll
    NLL += 1
    n1 += 1
    LL_1.append(t)
    tll_Data.append(t)
    t_ll = t + Generar_T_t()

    if n1 == 1:
      t_1 = t + Generar_Y()

  #Caso 2
  elif (t_1 < t_ll) and (t_1 <= t_2):
    t = t_1
    n1 -= 1
    n2 += 1
    LL.append(t)

    if n1 == 0:
      t_1 = infinity
    else:
      t_1 = t + Generar_Y()
    if n2 == 1:
      t_2 = t + Generar_Y2()

  #Caso 3
  elif (t_2 < t_1) and (t_2 < t_ll):
    t = t_2
    NS += 1
    n2 -= 1
    S.append(t)
    ts_Data.append(t)

    if n2 > 0:
      t_2 = t + Generar_Y2()
    else:
      t_2 = infinity

Calculamos el numero promedio de clientes en los servidores y el tiempo promedio que un cliente pasa en el sistema

In [43]:
T_final = t

salidas = min(len(tll_Data), len(ts_Data))
salidas_S1 = min(len(tll_Data), len(LL))

L1_sum = sum(n * duracion for n, duracion in servidor_1) if servidor_1 else 0
L2_sum = sum(n * duracion for n, duracion in servidor_2) if servidor_2 else 0

L1_promedio = L1_sum / T_final if T_final > 0 else 0
L2_promedio = L2_sum / T_final if T_final > 0 else 0
L_total_promedio = L1_promedio + L2_promedio

salida = min(len(tll_Data), len(ts_Data))

if salida > 0:
  T_promedio = np.array(ts_Data[:salidas]) - np.array(tll_Data[:salidas])
  W_t = np.mean(T_promedio)

  tiempos_en_servidor_2 = np.array(S[:salidas]) - np.array(LL[:salidas])
  W_t2 = np.mean(tiempos_en_servidor_2)

In [44]:
L_total_promedio

6.996650368861406

In [45]:
W_t

np.float64(1.394996668125135)

In [46]:
W_t2

np.float64(0.484095545292284)

Ahora con las formulas analiticas vamos a ver si es correcto

In [47]:
# Verificar estabilidad (condición: lambda_ll < mu_1 y lambda_ll < mu_2)
if lambda_ll >= mu_1 or lambda_ll >= mu_2:
    print("El sistema es inestable (lambda_ll_ll >= mu) para al menos un servidor. Las fórmulas de estado estacionario no aplican.")
else:
    # --- Servidor 1 (S1) ---
    L1_teorico = lambda_ll / (mu_1 - lambda_ll)
    W1_teorico = 1 / (mu_1 - lambda_ll)
    Lq1_teorico = (lambda_ll**2) / (mu_1 * (mu_1 - lambda_ll))
    Wq1_teorico = lambda_ll / (mu_1 * (mu_1 - lambda_ll))

    # --- Servidor 2 (S2) ---
    L2_teorico = lambda_ll / (mu_2 - lambda_ll)
    W2_teorico = 1 / (mu_2 - lambda_ll)
    Lq2_teorico = (lambda_ll**2) / (mu_2 * (mu_2 - lambda_ll))
    Wq2_teorico = lambda_ll / (mu_2 * (mu_2 - lambda_ll))

    # --- Sistema Total ---
    L_total_teorico = L1_teorico + L2_teorico
    W_total_teorico = W1_teorico + W2_teorico

In [48]:
L_total_promedio

6.996650368861406

In [49]:
W_total_teorico

1.5